## Coroutines

In [1]:
# Simplest coroutine
def simple_coroutine():
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)

my_coro = simple_coroutine()
my_coro

<generator object simple_coroutine at 0x109a26b88>

In [2]:
next(my_coro)

-> coroutine started


In [3]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

In [4]:
my_coro = simple_coroutine()
my_coro.send(1729)

TypeError: can't send non-None value to a just-started generator

In [7]:
my_coro.send(None)

-> coroutine started


In [8]:
my_coro.send(None)

-> coroutine received: None


StopIteration: 

In [10]:
my_coro.send(42)

StopIteration: 

In [11]:
# A coroutine that yields twice
def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

my_coro2 = simple_coro2(14)

In [12]:
from inspect import getgeneratorstate
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [13]:
next(my_coro2)

-> Started: a = 14


14

In [14]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [15]:
my_coro2.send(28)

-> Received: b = 28


42

In [16]:
my_coro2.send(99)

-> Received: c = 99


StopIteration: 

In [17]:
# To compute a running average
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

coro_avg = averager()
next(coro_avg)
coro_avg.send(10)

10.0

In [18]:
coro_avg.send(30)

20.0

In [19]:
coro_avg.send(5)

15.0

In [20]:
coro_avg.close()

In [ ]:
# decorator for priming coroutine
# coroutil.py
from functools import wraps

def coroutine(func):
    """Decorator: primes 'func' by advancing to first 'yield'"""
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

In [31]:
from coroutil import coroutine

@coroutine
def averager1():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

coro_avg1 = averager1()
coro_avg1.send(10)

10.0

In [32]:
coro_avg1.send(30)

20.0

In [34]:
coro_avg1.send(5)

15.0

In [35]:
coro_avg1.send('bad')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [38]:
# test code for studying exception handling in a coroutine
# coro_exc_demo.py
class DemoException(Exception):
    """An exception type for the demonstration"""
    
def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.')

In [39]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [40]:
exc_coro.send(11)

-> coroutine received: 11


In [41]:
exc_coro.send(22)

-> coroutine received: 22


In [42]:
exc_coro.close()

In [43]:
from inspect import getgeneratorstate
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [44]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [45]:
exc_coro.send(11)

-> coroutine received: 11


In [46]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [47]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [48]:
exc_coro.close()

In [49]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [50]:
exc_coro.send(11)

-> coroutine received: 11


In [51]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [52]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [53]:
# coro_finally_demo.py  try/finally
class DemoException(Exception):
    """An exception type for the demonstration"""
    
def demo_finally():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...')
            else:
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

In [54]:
# coroaverager2.py return a result
from collections import namedtuple

Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [60]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(5)
coro_avg.send(None)

StopIteration: Result(count=3, average=15.0)

In [61]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(5)
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
result

Result(count=3, average=15.0)

### Using yield from

In [2]:
# yield
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i

list(gen())

['A', 'B', 1, 2]

In [3]:
# yield from
def gen():
    yield from 'AB'
    yield from range(1, 3)

list(gen())

['A', 'B', 1, 2]

In [5]:
# coroaverager3.py using yield from to drive averager and report statistic
from collections import namedtuple

Result = namedtuple('Result', 'count average')

#the subgenerator
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

#the delegating generator
def grouper(result, key):
    while True:
        result[key] = yield from averager()
        
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)
        for value in values:
            group.send(value)
        group.send(None)  #important!
        
    report(results)
    
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(
              result.count, group, result.average, unit))

data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]
}

if __name__ == '__main__':
    main(data)

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


### Meaning of yield from

In [ ]:
RESULT = yield from EXPR

# simplified pseudocode
_i = iter(EXPR)
try: 
    _y = next(_i)
except StopIteration as _e:
    _r = _e.value
else:
    while 1:
        _s = yield _y
        try:
            _y = _i.send(_s)
        except StopIteration as _e:
            _r = _e.value
            break
            
RESULT = _r

_i subgenerator
_y yielded from the subgenerator
_r result
_s send
_e exception

In [ ]:
## Pseudocode to RESULT = yield from EXPR
_i = iter(EXPR)
try:
    _y = next(_i)
except StopIteration as _e:
    _r = _e.value
else:
    while 1:
        try:
            _s = yield _y
        except GeneratorExit as _e:
            try:
                _m = _i.close
            except AttributeError:
                pass
            else:
                _m()
            raise _e
        except BaseException as _e:
            _x = sys.exc_info()
            try:
                _m = _i.throw
            except AttributeError:
                raise _e
            else:
                try:
                    _y = _m(*_x)
                except StopIteration as _e:
                    _r = _e.value
                    break
        else:
            try:
                if _s is None:
                    _y = next(_i)
                else:
                    _y = _i.send(_s)
            except StopIteration as _e:
                _r = _e.value
                break
    
RESULT = _r

### The taxi fleet simulation

In [86]:
# taxi_sim.py
from collections import namedtuple
import queue
import time
import random

SEARCH_DURATION = 5
TRIP_DURATION = 20
DEPARTURE_INTERVAL = 5

Event = namedtuple('Event', 'time proc action')

def taxi_process(ident, trips, start_time=0):
    """yield to simulator issuing event at each state change"""
    time = yield Event(start_time, ident, 'leave garage')
    for i in range(trips):
        time = yield Event(time, ident, 'pick up passenger')
        time = yield Event(time, ident, 'drop off passenger')
    
    yield Event(time, ident, 'going home')
    
class Simulator:
    
    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)
        
    def run(self, end_time):
        """Schedule and display events until time is up"""
        for _, proc in sorted(self.procs.items()):
            first_event = next(proc)
            self.events.put(first_event)
            
        #main loop of the simulation
        sim_time = 0 
        while sim_time < end_time:
            if self.events.empty():
                print('*** end of events ***')
                break
            
            current_event = self.events.get()
            sim_time, proc_id, previous_action = current_event
            print('taxi:', proc_id, proc_id * ' ', current_event)
            active_proc = self.procs[proc_id]
            next_time = sim_time + compute_duration(previous_action)
            try:
                next_event = active_proc.send(next_time)
            except StopIteration:
                del self.procs[proc_id]
            else:
                self.events.put(next_event)
        else:
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))
            
def compute_duration(previous_action):
    """Compute action duration using exponential distribution"""
    if previous_action in ['leave garage', 'drop off passenger']:
        interval = SEARCH_DURATION
    elif previous_action == 'pick up passenger':
        interval = TRIP_DURATION
    elif previous_action == 'going home':
        interval = 1
    else:
        raise ValueError('Unknown previous_action: %s' % previous_action)
    return int(random.expovariate(1/interval)) + 1
    
#from taxi_sim import taxi_process
taxi = taxi_process(ident=13, trips=2, start_time=0)
next(taxi)

Event(time=0, proc=13, action='leave garage')

In [63]:
taxi.send(_.time + 7)

Event(time=7, proc=13, action='pick up passenger')

In [65]:
taxi.send(_.time + 1)

Event(time=15, proc=13, action='pick up passenger')

In [67]:
taxi.send(_.time + 12)

Event(time=27, proc=13, action='drop off passenger')

In [68]:
taxi.send(_.time + 12)

Event(time=39, proc=13, action='going home')

In [69]:
taxi.send(_.time + 12)

StopIteration: 

In [87]:

num_taxis = 3
taxis = {i: taxi_process(i, (i + 1) * 2, i * DEPARTURE_INTERVAL) 
        for i in range(num_taxis)}
sim = Simulator(taxis)
sim.run(100)

taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 1   Event(time=5, proc=1, action='leave garage')
taxi: 1   Event(time=6, proc=1, action='pick up passenger')
taxi: 0  Event(time=8, proc=0, action='pick up passenger')
taxi: 2    Event(time=10, proc=2, action='leave garage')
taxi: 0  Event(time=15, proc=0, action='drop off passenger')
taxi: 2    Event(time=15, proc=2, action='pick up passenger')
taxi: 0  Event(time=18, proc=0, action='pick up passenger')
taxi: 1   Event(time=23, proc=1, action='drop off passenger')
taxi: 2    Event(time=28, proc=2, action='drop off passenger')
taxi: 0  Event(time=29, proc=0, action='drop off passenger')
taxi: 1   Event(time=29, proc=1, action='pick up passenger')
taxi: 0  Event(time=33, proc=0, action='going home')
taxi: 2    Event(time=34, proc=2, action='pick up passenger')
taxi: 1   Event(time=39, proc=1, action='drop off passenger')
taxi: 1   Event(time=40, proc=1, action='pick up passenger')
taxi: 2    Event(time=42, proc=2, action='drop 